In [ ]:
import pandas as pd

In [150]:
# Загружены тренировочные и тестовые данные
data_train = pd.read_csv('rusentitweet_train.csv')
data_test = pd.read_csv('rusentitweet_test.csv')

# Оставлены только строки с label 'positive' или 'negative'
data_train = data_train[data_train['label'].isin(['positive', 'negative'])]
data_test = data_test[data_test['label'].isin(['positive', 'negative'])]

In [156]:
import re

# Функция для очистки текста от веб-ссылок, ссылок на твиттер-аккаунты и хэштегов
def clean_data(text):
  text = re.sub(r'(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)', '', text) # Убираем веб-ссылки
  text = re.sub(r'@[^\s]+', '', text) # Убираем ссылки на твиттер-аккаунты
  text = re.sub(r'#\w+', '', text) # Убираем хэштеги
  return text

# Применение функции для очистки к столбцу text тренировочного и тестового датафреймов
data_train['text'] = data_train['text'].apply(clean_data)
data_test['text'] = data_test['text'].apply(clean_data)

data_train.head()

,text,label,id
0,Помойму я вкрашилась в Чимина🤧,positive,1282311169534038016
5,я не с порядке!!!!,negative,1335130757044563971
10,В следующий раз буду до победного ее закрыват...,positive,1215370396465291267
15,"Удивительный гiмн...\r\nУдивительно, что пока...",negative,1253799540848762887
17,я срала на эту биологию,negative,1339418979887173632


In [ ]:
from nltk.stem.snowball import SnowballStemmer

# Инициализация объекта-стеммера
stemmer = SnowballStemmer('russian')

# Функция для стемминга
def stemming(text):
  return ' '.join([stemmer.stem(s) for s in text.split()])

# Применение стемминга
stemmed_data_train = data_train.copy()
stemmed_data_test = data_test.copy()
stemmed_data_train['text'] = stemmed_data_train['text'].apply(stemming)
stemmed_data_test['text'] = stemmed_data_test['text'].apply(stemming)

stemmed_data_train.head()

,text,label,id
0,помойм я вкраш в чимина🤧,positive,1282311169534038016
5,я не с порядке!!!!,negative,1335130757044563971
10,в след раз буд до победн е закрыва пожела удачи!,positive,1215370396465291267
15,"удивительн гимн... удивительно, что пок ещ не ...",negative,1253799540848762887
17,я срал на эт биолог,negative,1339418979887173632


In [168]:
from pymystem3 import Mystem

# Инициализация объекта для лемматизации
mystem = Mystem()

# Функция лемматизации
def lemmatization(text):
  lemmas = mystem.lemmatize(text)
  res = [l for l in lemmas if l.isalnum()]
  return ' '.join(res)

# Применение лемматизации
lemmed_data_train = data_train.copy()
lemmed_data_test = data_test.copy()
lemmed_data_train['text'] = lemmed_data_train['text'].apply(lemmatization)
lemmed_data_test['text'] = lemmed_data_test['text'].apply(lemmatization)

lemmed_data_train.head()

,text,label,id
0,помойма я вкрашиться в чимин,positive,1282311169534038016
5,я не с порядок,negative,1335130757044563971
10,в следующий раз быть до победный она закрывать...,positive,1215370396465291267
15,удивительный гiмн удивительно что пока еще не ...,negative,1253799540848762887
17,я срать на этот биология,negative,1339418979887173632


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Инициализация объекта для преобразования в мешок
vectorizer = CountVectorizer()

# Осуществление преобразования
data_train_count_M = vectorizer.fit_transform(lemmed_data_train['text'])
data_test_count_M = vectorizer.transform(lemmed_data_test['text'])

print(data_train_count_M)

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

# Инициализация объекта для расчета tf-idf
transformer = TfidfTransformer()

# Осуществление расчета tf-idf
data_train_tfidf = transformer.fit_transform(data_train_count_M)
data_test_tfidf = transformer.transform(data_test_count_M)

print(data_train_tfidf)

In [171]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Превращение слов positive и negative в 1 и 0
label_convert = {'positive': 1, 'negative': 0}
data_train_targets = data_train['label'].map(label_convert)
data_test_targets = data_test['label'].map(label_convert)

# Создание моделей логистической регрессии и случайного леса
logistic_reg = LogisticRegression()
random_forest = RandomForestClassifier()

# Обучение моделей
logistic_reg.fit(data_train_tfidf, data_train_targets)
random_forest.fit(data_train_tfidf, data_train_targets)

# Проверка на тестовых данных
logistic_reg_predict = logistic_reg.predict(data_test_tfidf)
random_forest_predict = random_forest.predict(data_test_tfidf)

# Проверка на тренировочных данных
logistic_reg_predict_train = logistic_reg.predict(data_train_tfidf)
random_forest_predict_train = random_forest.predict(data_train_tfidf)

In [172]:
from sklearn.metrics import classification_report

# Отчет по работе модели логистической регрессии на тестовых данных
print('\tЛогистическая регрессия\n\n', classification_report(data_test_targets, logistic_reg_predict))

# Отчет по работе модели случайного леса на тестовых данных
print('\tСлучайный лес\n\n', classification_report(data_test_targets, random_forest_predict))

	Логистическая регрессия

               precision    recall  f1-score   support

           0       0.76      0.92      0.83       660
           1       0.85      0.60      0.71       483

    accuracy                           0.79      1143
   macro avg       0.81      0.76      0.77      1143
weighted avg       0.80      0.79      0.78      1143

	Случайный лес

               precision    recall  f1-score   support

           0       0.76      0.81      0.78       660
           1       0.71      0.65      0.68       483

    accuracy                           0.74      1143
   macro avg       0.74      0.73      0.73      1143
weighted avg       0.74      0.74      0.74      1143



In [173]:
from sklearn.metrics import classification_report

# Отчет по работе модели логистической регрессии на тренировочных данных
print('\tЛогистическая регрессия\n\n', classification_report(data_train_targets, logistic_reg_predict_train))

# Отчет по работе модели случайного леса на тренировочных данных
print('\tСлучайный лес\n\n', classification_report(data_train_targets, random_forest_predict_train))

	Логистическая регрессия

               precision    recall  f1-score   support

           0       0.88      0.97      0.92      2638
           1       0.96      0.81      0.88      1931

    accuracy                           0.91      4569
   macro avg       0.92      0.89      0.90      4569
weighted avg       0.91      0.91      0.91      4569

	Случайный лес

               precision    recall  f1-score   support

           0       1.00      1.00      1.00      2638
           1       1.00      1.00      1.00      1931

    accuracy                           1.00      4569
   macro avg       1.00      1.00      1.00      4569
weighted avg       1.00      1.00      1.00      4569



In [174]:
# Получение имен признаков
feature_names = vectorizer.get_feature_names_out()

# Получение и сортировка абсолютных значений коэффициентов модели лог. регрессии
lr_coefs = logistic_reg.coef_[0]
lr_coefs = abs(lr_coefs).argsort()[::-1]

# 20 самых важных слов для модели лог. регрессии
lr_most_important_features = [feature_names[i] for i in lr_coefs[:20]]
print('20 самых важных слов для модели лог. регрессии:\n', lr_most_important_features)

print('')

# Получение и сортировка коэффициентов модели случайного леса
rf_feature_importanсes = random_forest.feature_importances_
rf_feature_importanсes = rf_feature_importanсes.argsort()[::-1]

# 20 самых важных слов для модели случайного леса
rf_most_important_features = [feature_names[i] for i in rf_feature_importanсes[:20]]
print('20 самых важных слов для модели случайного леса:\n', rf_most_important_features)

20 самых важных слов для модели лог. регрессии:
 ['любить', 'не', 'хороший', 'блять', 'пиздец', 'красивый', 'нет', 'нравиться', 'вау', 'нахуй', 'любовь', 'прекрасный', 'сука', 'милый', 'умирать', 'классный', 'крутой', 'рад', 'хорошо', 'круто']

20 самых важных слов для модели случайного леса:
 ['не', 'любить', 'хороший', 'ты', 'это', 'что', 'блять', 'нет', 'так', 'пиздец', 'мой', 'красивый', 'такой', 'все', 'вау', 'хорошо', 'нравиться', 'быть', 'на', 'как']
